In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

In [2]:
data = pd.read_csv('../Corpus/normaliced_data/politicES.csv', index_col=0)

X = data['tweets']

y = data['genero']

print(X.shape)
print(y.shape)

(180000,)
(180000,)


In [3]:
# Se divide el conjunto de entrenamiento y de orueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

print('X entrenamiento: {}, y entrenamiento: {}\nX prueba: {}, y prueba: {}'.format(X_train.shape,y_train.shape, X_test.shape, y_test.shape))

X entrenamiento: (144000,), y entrenamiento: (144000,)
X prueba: (36000,), y prueba: (36000,)


y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [4]:
# Se crea una instancia de TfidfVectorizer() y se asigna a la variable vectorized
vectorized = TfidfVectorizer()
# x_train_tfidf se obtiene al aplicar el método fit_transform() de vectorized a los datos de entrenamiento x_train
x_train_tfidf = vectorized.fit_transform(X_train)
x_test_tfidf = vectorized.transform(X_test)

print('TF-IDF:\nX entrenamiento: {} X prueba: {}'.format(x_train_tfidf.shape, x_test_tfidf.shape))

TF-IDF:
X entrenamiento: (144000, 77038) X prueba: (36000, 77038)


In [5]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

X_train_oversampled_genero, y_train_oversampled_genero = smote.fit_resample(x_train_tfidf, y_train)
print(X_train_oversampled_genero.shape)
print(y_train_oversampled_genero.shape)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fdc52706200>
Traceback (most recent call last):
  File "/home/bruno-rg/anaconda3/envs/nlp/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/bruno-rg/anaconda3/envs/nlp/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/bruno-rg/anaconda3/envs/nlp/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/bruno-rg/anaconda3/envs/nlp/lib/python3.10/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'


(191098, 77038)
(191098,)


In [8]:
from imblearn.under_sampling import RandomUnderSampler
# Aplicar undersampling
undersampler = RandomUnderSampler(random_state=42)

X_undersampled_genero, y_undersampled_genero = undersampler.fit_resample(x_train_tfidf, y_train)
print(X_undersampled_genero.shape)
print(y_undersampled_genero.shape)

(96902, 77038)
(96902,)


In [6]:
def Regresion(solver, penalty):
    lr_model = LogisticRegression(penalty=penalty, solver=solver, max_iter=500)
    # Se ajustan los datos al modelo
    lr_model.fit(X_train, y_train)

    predictions = lr_model.predict(X_test)

    # Calcular la exactitud
    print(solver)
    print(penalty)
    exactitud = round(accuracy_score(y_test, predictions),2)
    presicion = round(precision_score(y_test, predictions),2)
    f1 = round(f1_score(y_test, predictions),2)
    recall = round(recall_score(y_test, predictions),2)

    print("Exactitud:", exactitud)
    print("Exactitud:", presicion)
    print("Exactitud:", f1)
    print("Exactitud:", recall)

'\nlr_model = LogisticRegression(max_iter=500)\n# Se ajustan los datos al modelo\nlr_model.fit(x_train_tfidf, y_train)\n# Se utiliza el modelo ya ajustado para predecir los tatos de prueba y se evalua\n#lr_score = lr_model.score(x_test_tfidf,y_test)\n# Se imprime la presición del modelo\n#print(f\'Accuracy Regresión Logística: {lr_score}\')\n\n#Se crea un objeto KFold con 5 divisiones (n_splits=5) que se utilizará para dividir los datos en 5 conjuntos diferentes\nkf = KFold(n_splits=5, shuffle=True, random_state=42)\n\n# Se aplica la validación cruzada al modelo de regresión logística\nlr_scores = cross_val_score(lr_model, x_train_tfidf, y_train, cv=kf)\n# Imprimimos los resultados de la validación cruzada\nprint(f"k-fold cross-validation LR: {np.mean(lr_scores)*100:.4f}%")'